### Print-hack (spring over)

Først laver vi først et lille hack ved at lave en `Print` funktion, som kan udskrive resultater i Jupyter Notebook.  Det skal du ikke gøre i din kode i Visual Studio.

In [17]:
string Print(object output)
{
    return output.ToString();
}

Vi kan bruge Print til at udskrive diverse ting.

In [18]:
Print("Hej med dig!");

Hej med dig!

# Indlæsning af filer i C#
Der er flere måder at indlæse filer på i C#.  Herunder finder du en beskrivelse af, hvordan man kan indlæse en tekstfil en linje ad gangen.

## StreamReader
.NET tilbyder flere klasser til filhåndtering.  Den klasse, vi skal bruge, hedder [`StreamReader`][1] og findes i det namespace, der hedder `System.IO`.  Start derfor med at inkludere `System.IO` øverst i din .cs fil.

[1]: https://msdn.microsoft.com/en-us/library/system.io.streamreader(v=vs.110).aspx


In [19]:
using System.IO;

Lad os starte med at oprette en `StreamReader`.  Her skal du angive stien til den fil, som du vil indlæse.  Samtidig er det en god idé at angive encoding, så danske bogstaver og andre sjove tegn bliver indlæst korrekt.

Når man indlæser en fil, låser Windows filen samtidig.  Der skal du huske at frigive filen, når du er færdig med den.  `StreamReader` har en funktion, der hedder `Dispose`, som frigiver filen.  `Dispose` er defineret på et interface, der hedder `IDisposable`.  Når en klasse gør det, kan man putte objektet inden i en `using` blok, og `Dispose` vil så blive kaldt automatisk.

Når filen er åbnet, kan du læse en linje ad gangen med funktionen `ReadLine`.  Samtidig skal du bruge en anden funktion kaldet `Peek` til hele tiden at sikre, at du ikke har nået slutningen af filen.  `Peek` returner -1, hvis man er nået til slutningen af filen.

Det hele ser således ud.

In [20]:
var linjer = new List<string>();
using (var reader = new StreamReader("biler.csv", Encoding.Default))
{
    while (reader.Peek() > -1)
    {
        var linje = reader.ReadLine();
        linjer.Add(linje);
    }
}

Print("Antal linjer: " + linjer.Count);


Antal linjer: 4

Ovenfor tilføjede du teksten fra hver linje til en liste af strenge (`List<string>`), men i den virkelige verden vil man gerne kunne bruge den indlæste fil til noget fornuftigt.  Indholdet af filen ser således ud.

Id;Navn;Bilmærke  
1;Lasse;Renault  
2;Marcus;Chevrolet  
3;Per;Audi  

Her har vi fat i en komma-separeret fil (med semikolon som separatator).  For at bruge sådan en fil fornuftigt, vil vi skulle springe overskriften over og splitte linjer ved semikolon.  CSV-filer kommer i alle afskygninger, så derfor har vi i AT lavet en generel klasse, som er god til netop det, kaldet `CsvReader`.

## CsvReader
`CsvReader` er del af en større pakke, som ligger i AT's Nuget feed (spørg Jakob).  I dette dokument kan vi ikke bruge Nuget, så vi indlæser dll'en med CsvReader manuelt (det skal du ikke gøre i din kode).


In [21]:
// Gør ikke dette i Visual Studio!
#r "AP.Core.dll" 

Tilføj derefter det namespace, som `CsvReader` ligger i.

In [22]:
using AP.FilImport.FilVaerktoejer;

Før vi tager `CsvReader` i brug, skal vi først lave en klasse, som vi kan bruge til at holde det data, vi indlæser fra filen.  Dvs. vi skal have en klasse, som kan holde id, navn og bilmærke.

In [23]:
public class Bil
{
    public Bil(int id, string navn, string bilmærke)
    {
        this.Id = id;
        this.Navn = navn;
        this.Bilmærke = bilmærke;
    }
    
    public int Id { get; private set; }
    
    public string Navn { get; private set; }
    
    public string Bilmærke { get; private set; }
}

Med den nye Bil-klasse, er du nu klar til at bruge `CsvReader`.  Det svarer meget til at bruge `StreamReader`, men du har f.eks. mulighed for at angive separator og bruge overskrifterne i CSV-filen.  Du kan også springe nogle linjer i starten af filen over, hvis det er nødvendigt.

In [24]:
// Opret liste af biler til at holde de biler, vi indlæser.
var biler = new List<Bil>();
using (var reader = new CsvReader("biler.csv", skipNumberOfLines: 0, firstLineContainsColumnNames: true))
{
    while (reader.Read())
    {
        var bil = new Bil(
            Convert.ToInt32(reader["Id"]),
            reader["Navn"],
            reader["Bilmærke"]
        );
        
        biler.Add(bil);
    }
}

Print("Antal biler: " + biler.Count);

Antal biler: 3

In [25]:
Print($"{biler[0].Navn}'s bil er en {biler[0].Bilmærke}.");

Lasse's bil er en Renault.

In [26]:
// Hjælpefunktion til Jupyter Notebook. Brug ikke i Visual Studio!
string Print(object output)
{
    return output.ToString();
}

# Tilgang til Microsoft SQL Server i C# #
Hermed en kort beskrivelse af, hvordan du kan læse fra og skrive data til en SQL Server database fra C#.


I .NET ligger klasserne til at tilgå SQL Server i to namespaces, som du skal starte med at inkludere øverst i din .cs fil.

In [27]:
using System.Data;
using System.Data.SqlClient;

Næste skridt er at fortælle C#, hvilken SQL Server og database, vi ønsker at bruge.  Det gør man vha. en såkaldt connection string, som ser således ud.

In [28]:
string connectionString = @"Integrated Security=SSPI;Persist Security Info=False;Initial Catalog=datamartnavn;Data Source=sqlinstansnavn";

Denne connection string fortæller, at SQL serverinstancen hedder "sqlinstansnavn", databasen hedder"datamartnavn", og du ønsker at logge ind på SQL serveren med dit almindelige Windows login.

Login til databasen sker vha. en klasse, der hedder `SqlConnection`.  Når man bruger `SqlConnection`, skal man huske altid at lukke igen (`Dispose`).  Derfor pakker vi den ind i en `using` block.  Som parameter tager `SqlConnection` vores connection string.

In [29]:
using (var cn = new SqlConnection(connectionString))
{
    cn.Open();
}

Der opstod en netværksrelateret eller forekomstspecifik fejl, da det blev forsøgt at oprette forbindelse til SQL Server. Serveren blev ikke fundet, eller der var ikke adgang til den. Kontroller, at forekomstnavnet er korrekt, og at SQL Server er konfigureret til at tillade fjernforbindelser. (provider: Named Pipes Provider, error: 40 - Der kunne ikke åbnes en forbindelse til SQL Server.)

Indtil videre er der ikke så meget sjov ved vores connection, så lad os afvikle noget SQL med en SELECT statement.  Det gør man gennem klassen `SqlCommand`.  `SqlCommand` har en metode med navnet `ExecuteReader`, som returnerer et object af type `SqlDataReader`, der kan bruges til at løbe resultatet en en SELECT statement igennem linje for linje.  En reader skal også altid lukkes med `Dispose`.

In [30]:
var output = new StringBuilder();
using (var cn = new SqlConnection(connectionString))
{
    var cmd = new SqlCommand("SELECT TOP(10) PoliceNummer FROM Dimensions.Police", cn);
    cn.Open();
    
    using (var reader = cmd.ExecuteReader())
    {
        while (reader.Read())
        {
            output.AppendLine((string)reader["PoliceNummer"]);
        }
    }
}

Print(output.ToString());

Der opstod en netværksrelateret eller forekomstspecifik fejl, da det blev forsøgt at oprette forbindelse til SQL Server. Serveren blev ikke fundet, eller der var ikke adgang til den. Kontroller, at forekomstnavnet er korrekt, og at SQL Server er konfigureret til at tillade fjernforbindelser. (provider: Named Pipes Provider, error: 40 - Der kunne ikke åbnes en forbindelse til SQL Server.)

Bemærk hvordan vi løber resultatet igennem linje for linje med `while` løkken, og for hver linje beder vi om værdien i "PoliceNummer"-kolonnen.

Måske ønsker vi kun at se nogle oplysninger for en enkelt police, så vi vil gerne kunne parametrisere vores SELECT med en WHERE clause.  En mulighed er at skrive "SELECT ... WHERE PoliceNummer = 1705490401'". Men hvis du har brugt for at køre SQL'en flere gange for forskellige policenumre, er det rigtige at gøre at bruge SQL parametre som vist herunder.  Der er flere fordele ved det.  En af dem er, at formatet f.eks. for datoer altid bliver korrekt.  Det sørger SQL server for.  Sådan ser den <strong>rigtige</strong> måde ud:

In [31]:
var output = new StringBuilder();
using (var cn = new SqlConnection(connectionString))
{
    var cmd = new SqlCommand("SELECT TOP(10) PoliceNummer, PoliceStatus FROM Dimensions.Police WHERE PoliceNummer = @policeNummer", cn);
    cmd.Parameters.Add("@policeNummer", SqlDbType.NVarChar, 20).Value = "1705490401";
    cn.Open();

    using (var reader = cmd.ExecuteReader())
    {
        while (reader.Read())
        {
            output.AppendLine((string)reader["PoliceNummer"]);
            output.AppendLine((string)reader["PoliceStatus"]);
        }
    }
}
            
Print(output.ToString());


Der opstod en netværksrelateret eller forekomstspecifik fejl, da det blev forsøgt at oprette forbindelse til SQL Server. Serveren blev ikke fundet, eller der var ikke adgang til den. Kontroller, at forekomstnavnet er korrekt, og at SQL Server er konfigureret til at tillade fjernforbindelser. (provider: Named Pipes Provider, error: 40 - Der kunne ikke åbnes en forbindelse til SQL Server.)

Du kan naturligvis også afvikle andre typer SQL, som f.eks. UPDATE, INSERT og DELETE.  Til det formål skal du bruge `ExecuteNonQuery` i stedet for `ExecuteReader`. `ExecuteNonQuery` returnerer antal rækker, der er påvirket af SQL'en.  For INSERT vil metoden f.eks. returnere antal nye rækker.

In [35]:
int antalRækker = 0;
using (var cn = new SqlConnection(@"Integrated Security=SSPI;Persist Security Info=False;Initial Catalog=Test;Data Source=.\SQLEXPRESS"))
{
    // Sådan vil man normalt ikke lave to INSERTs.
    var cmd = new SqlCommand(@"
        INSERT INTO TestTable (Id, TestKolonne) VALUES (@id1, @testVærdi1)
        INSERT INTO TestTable (Id, TestKolonne) VALUES (@id2, @testVærdi2)
        ", cn);
        
    cmd.Parameters.Add("@id1", SqlDbType.Int).Value = 42;
    cmd.Parameters.Add("@testVærdi1", SqlDbType.NVarChar, 20).Value = "abcde";
    cmd.Parameters.Add("@id2", SqlDbType.Int).Value = 43;
    cmd.Parameters.Add("@testVærdi2", SqlDbType.NVarChar, 20).Value = "fghij";
    cn.Open();

    antalRækker = cmd.ExecuteNonQuery();
}
            
Print(antalRækker);


2

# Hvordan gemmer man lange SQL statements i SQL?

En måde at gemme lange stykker SQL kode i sit C# projekt, er at gemme dem som en ressource i selve projektet.  En ressource kan være en streng, en fil, et billede eller andet, som bliver "embedded" i ens .exe eller .dll fil, og man kan tilgå ressourcen fra kode.

Først skal vi sikre os, at projektet har en ressource fil.  Højreklik på projektet i Solution Explorer i Visual Studio og vælg Properties.  I højre side er der et punkt, der hedder Resources.  Hvis projektet ikke har en ressource fil i forvejen, vil der være et link, hvor der står "This project does not contain a default resources file. Click here to create one".

<img src="files/Images/vs-add-ressource-file.png">

Når du klikker på linket, vil VS tilføje en ny fil kaldet Resources.resx i dit projekt.

<img src="files/Images/vs-ressources-file.PNG">

Næste trin er at lave dine SQL filer.  Man laver en fil pr. SQL statement og tilføjer til VS projektet.  Det er en god idé at placere filerne i en underfolder i projektet:

1. Højre-klik på dit projekt i Solution Explorer og vælg Add -> New Folder.
1. Kald den nye foldere SQL eller Ressources eller noget andet sigende.
1. Højre-klik på folderen og vælge Add -> New Item.
1. Søg efter "text" og vælg Text File.
1. Indtast et navn for filen med .sql extension og tryk Add.

    <img src="files/Images/vs-add-sql-file.png" >
        
1. Skriv din SQL statement i den nye .sql fil og gem.
1. Dobbelt-klik på din Resources.resx fil i Solution Explorer.
1. Tryk på Add Resource knappen øverst og vælg Add Existing File fra dropdown menuen.

    <img src="files/Images/vs-ressources-add-file.PNG" >

1. Browse til din nye .sql fil og tilføj den.

Når du gemmer Resource.resx vil VS lave lidt magi, som gør det meget nemt at få fat i SQL'en fra C#.  VS laver automatisk en Properties klasse, som man kan bruge:

```C#
var cmd = new SqlCommand(Properties.Resources.MySql, connection);
```


# Unit tests

Unit tests er kode, som tester kode.  Dvs. du skal skrive kode, hvis formål er at teste, at din egentlige programkode opfører sig som forventet.  Det ligger i betegnelsen, at formålet er at teste afgrænsede enheder (units) af koden.  Så formålet er ikke at teste, at hele programmet fungerer samlet set.  Der skal man bruge regressions- og integrationstest.

Målet med unit tests er at de skal give udvikleren en sikkerhed for, at hvis han eller hun retter i koden, fungerer den stadig efter hensigten.

Der er delte meninger om, hvad man bør teste med unit tests.  Nogle mener man skal teste mod testdatabaser og andet, mens andre mener, at unit tests skal kunne afvikles udelukkende in-memory.  Der er enighed om, at unit tests skal være hurtige, og de skal kunne afvikles automatisk.  F.eks. kan man konfigurere TFS til at afvikle unit tests automatisk, når man tjekker sin kode ind.

En unit test kunne f.eks. se således ud.  Her tester vi, at metoden Add kan lægge to tal rigtigt sammen.

```C#
int actual = Beregner.Add(1, 2);
Assert.AreEqual(3, actual);
```

Koden til unit tests er ikke en del af dit egentlige program, så derfor skal unit tests ligge et separat projekt men i samme solution som dit program i Visual Studio.  Højreklik derfor på din solution øverst i Solution Explorer og vælg Add -> New Project.  Vælg den projektskabelon der hedder "Class Library (.NET Framework)".

Unit test projekter skal du navngive som projektnavnet på det projekt, der skal testes, med ".UnitTest" bagefter, som vist på skærmbilledet herunder.  Her hedder vores program "Beregningsmaskine", så unit test projektet skal hedde "Beregningsmaskine.UnitTest".

Når det nye projekt er oprettet, skal du slette den automatisk oprettede fil Class1.cs fra projektet.

![Nyt projekt](Images/unittest-new-project.png)

## Installer XUnit

Til at unit teste bruger vi i AT et framework, der hedder [XUnit](https://xunit.github.io/).  XUnit sørger for at afvikle vores unit tests og fortælle, hvis nogen af dem går galt.  For at installere XUnit, skal du bruge Nuget, og for eksemplets skyld vil vi installere XUnit gennem den såkaldte Nuget package manager console.

> Unit test frameworks: Andre kendte unit test frameworks er NUnit og MSTest.

1. I VS gå til menuen View -> Other Windows -> Package Manager Console.  Dette vil åbne et vindue nederst i VS.
1. Vælg dit unit test projekt i "Default project" dropdown.
1. Skriv følgende kommando og tryk enter:
    ```
    install-package xunit
    ```
1. Gør det samme for:
    ```
    install-package xunit.runner.visualstudio
    ```

Du har nu installeret XUnit og en såkaldt Visual Studio runner for XUnit.  Denne runner gør det muligt nemt at afvikle unit tests fra VS.

For at teste koden i dit hovedprojekt (Beregningsmaskine i vores eksempel), skal du også tilføje en reference til det projekt fra dit unit test projekt.

1. I Solution Explorer, højreklik på References under dit unit test projekt og vælg Add Reference.
1. Klik på Projects til venstre.
1. Sæt et kryds ud for dit hovedprojekt.
1. Klik OK.

Referencen gør det muligt at kalde kode i hovedprojektet fra unit test projektet.  

> Acces modifiers: Da unit tests lever i et andet projekt en hovedkoden, kan unit tests som udgangspunkt kun teste public klasser og metoder.

## Kode til unit teste

Det er nu endelig tid til at skrive vores første unit test.  Antag at vores hovedprojekt har en klasse med én metode kaldet Division.  Det er tanken at Division skal kunne lave heltalsdivision, og det vil vi gerne unit teste.  Indtil videre kan vores metode ikke så meget.

```C#
public static class IntegerMath
{
    public static int Division(int tæller, int nævner)
    {
        return 0;
    }
}
```

En metode til at skrive unit tests er at skrive testen først og derefter rette koden til, så den opfylder testen. Denne metode kaldes test-drive development, og det er den metode, vi vil bruge til at teste vores Division funktion.

Tilføj en ny klasse til dit unit test projekt.  Kald klassen (og filen) for `IntegerMathTest`.  Øverst i den nye fil skal du tilføje følgende using statements:

```C#
using Xunit;
using Beregningsmaskine;
```

Tilføj en ny metode til `IntegerMathTest` klassen, som vist herunder.  Denne metode er vores første unit test.

```C#
public class IntegerMathTest
{
    [Fact]
    public void Division_PositiveTalNævnerGårOpITæller_ReturnererDivisor()
    {

    }
}
```

Bemærk den såkaldte attribut `[Fact]`.  Denne attribut fortæller XUnit, at denne metode er en unit test, som skal afvikles af XUnit, når vi kører vores unit tests.

> Navngivning af unit tests: Normalt bruger vi ikke underscore i type- og metodenavne, men unit tests er en undtagelse.  De navngives efter mønsteret [Metode der testes]\_[Hvad tester vi]\_[Forventet resultat].

Lad os skrive en test af, at metoden kan dividere 4 med 2.

```C#
[Fact]
public void Division_PositiveTalNævnerGårOpITæller_ReturnererDivisor()
{
    int tæller = 6;
    int nævner = 2;

    int actual = IntegerMath.Division(tæller, nævner);

    Assert.Equal(3, actual);       
}
```

Højreklik nu et sted inde i din test metode og vælg Run Tests.  VS vil nu åbne et nyt Test Explorer vindue af afvikle din test vha. XUnit.  Som forventet vil testen fejle, for vores funktion returnerer 0 og ikke 2 som forventet.

![Fejl i unit test](Images/unittest-fail.png)

Lad os prøve at rette metoden til, så den rent faktisk forsøger at regne noget.

```C#
public static int Division(int tæller, int nævner)
{
    return tæller / nævner;
}
```

Unit testen melder nu succes!

![Succesfuld unit test](Images/unittest-success.png)

Lad os tilføje en ny unit test, der tester, at funktionen kan håndtere heltalsdivision, hvor tæller ikke går op i nævner.  Vi ønsker, at funktionen returnerer den heltalsdivisor, der ligger tættest på nul.

```C#
[Fact]
public void Division_PositiveTalNævnerGårIkkeOpITæller_ReturnererDivisorTættestPåNul()
{
    int tæller = 7;
    int nævner = 2;

    int actual = IntegerMath.Division(tæller, nævner);

    Assert.Equal(3, actual);
}
```

Tryk Run All øverst i Test Explorer for at køre begge unit tests.  Som du kan se, kører begge unit tests igennem uden fejl.

> Heltalsdivision i C#: Hvorfor fejler vores anden unit test ikke?  

> Division med operatoren '/' i C# er som standard en heltalsdivision, hvis både tæller og nævner er heltal.  Derfor går vores unit test igennem uden fejl, uden at vi ændrede noget i koden (det betyder også, at vores Division metode er nytteløs, for C# har allerede heltalsdivision med '/').  

> Hvis du vil lave ikke-heltalsdivision, skal du sørge for, at nævneren er en double eller en float, f.eks. ved at lave en cast til double: 

> `double res = tæller / (double)nævner`.

Lad os tilføje en test af, at det virker, hvis tælleren er negativ.

```C#
[Fact]
public void Division_TællerNegativNævnerGårIkkeOpITæller_ReturnererDivisorTættestPåNul()
{
    int tæller = -7;
    int nævner = 2;

    int actual = IntegerMath.Division(tæller, nævner);

    Assert.Equal(-3, actual);
}
```

Igen går testen godt uden kodeændringer.

> Gentagelse af kode: Vi har nu skrevet 3 unit tests, som er meget ens.  Som for al anden kode, skal man helst undgå at gentage sig selv for at mindske byrden ved vedligeholdelse.  XUnit har en funktionalitet, der hedder theories, hvor man kan parametrisere unit tests.  Det vil vi ikke gennemgå her, men overlade det som en [øvelse til læseren](http://hamidmosalla.com/2017/02/25/xunit-theory-working-with-inlinedata-memberdata-classdata/).

Vi mangler at teste, hvad der sker, hvis nævneren er 0.  Hvis nævneren er 0, vil vi gerne have en exception.  Måden er man tester det på, er ved at bruge XUnit's `Assert.Throws` metode.

```C#
[Fact]
public void Division_TællerErIkke0NævnerEr0_Throws()
{
    int tæller = 7;
    int nævner = 0;

    Assert.Throws<Exception>(() => IntegerMath.Division(tæller, nævner));
}
```

Denne unit test fejler med beskeden _"Expected: typeof(System.Exception)
Actual:   typeof(System.DivideByZeroException"_. Det viser sig, at '/' kaster `DivideByZeroException`, hvis nævneren er 0, så vi ændrer unit testen.

```C#
Assert.Throws<DivideByZeroException>(() => IntegerMath.Division(tæller, nævner));
```

Metoden `Assert.Throws` benytter sig af en såkaldt delegate.  I vores tilfælde ser den delegate således ud.

```C#
() => IntegerMath.Division(tæller, nævner)
```

En delegate er en slags reference til en funktion, så man kan kalde en funktion indirekte.  Så vi giver XUnit en  reference til `IntegerMath.Division`, som XUnit så kalder for at tjekke, om der rent faktisk kommer en `DivideByZeroException`.

Vores sidste unit test i dette eksempel bliver for 0 divideret med 0.  I vanlig Matematisk Analyse ved studiet for Forsikringsvidenskab ved Københavns Universitet, synes vi, at 0 divideret med 0 skal give 1.

```C#
[Fact]
public void Division_TællerOgNævnerEr0_Returnerer1()
{
    int tæller = 0;
    int nævner = 0;

    int actual = IntegerMath.Division(tæller, nævner);

    Assert.Equal(1, actual);
}
```

Men testen fejler, fordi C# kaster en `DivideByZeroException`.  Så vi må fikse vores `Divide` funktion.

```C#
public static int Division(int tæller, int nævner)
{
    if (tæller == 0 && nævner == 0)
    {
        return 1;
    }

    return tæller / nævner;
}
```

Så kører alle tests igennem uden fejl.


## Omfang af unit tests

Der er lige så mange holdninger til omfanget af unit tests, som der er udviklere.  Nogle mener, at al kode skal afprøves igennem unit tests (såkaldt "100% code coverage").

I AT har vi en mere pragmatisk holdning til unit tests.  Vi mener, de skal teste forretningsregler, men ikke nødvendigs alle kombinationer af parametre.  Man skal også teste hjørnetilfælde, f.eks. division med 0 i ovenstående eksempel.

Unit tests bør altid skrives af den udvikler, der har skrevet den oprindelige